In [ ]:
import sys
import pandas as pd
from pathlib import Path
import numpy as np
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import contextily as cx

# Add project root to Python path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

import src.config as config

plt.ioff()

In [ ]:
from src.loadProcessed import load_sensor_effects

sensor_effects = load_sensor_effects()

In [ ]:

from pyproj import Transformer
import matplotlib.patches as patches
from matplotlib.colors import TwoSlopeNorm

plot_data = sensor_effects.copy()
plot_data['magnitude'] = plot_data['percentage_change'].abs()

# Convert lat/lon to Web Mercator (meters) for Contextily
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
plot_data['x'], plot_data['y'] = transformer.transform(plot_data['longitude'].values, plot_data['latitude'].values)

fig, ax = plt.subplots(figsize=(6, 8))

# Create Colormap with Two-Slope Normalization
cmap = mpl.colormaps['RdYlGn']

# Two-slope norm centers at 0
traffic_change_norm = TwoSlopeNorm(
    vmin=plot_data['percentage_change'].min(),
    vcenter=0,
    vmax=plot_data['percentage_change'].max()
)

# Scatter points
sc = ax.scatter(
    plot_data['x'], plot_data['y'],
    cmap=cmap, 
    c=plot_data['percentage_change'],
    norm=traffic_change_norm,
    edgecolors='white',
    s= 50 * np.sqrt(plot_data['magnitude']),
    alpha=.9,
    zorder=2
)

# Add colourbar
cbar = fig.colorbar(sc, ax=ax, orientation='horizontal', 
                    fraction=0.03, pad=0.01, aspect=30)
cbar.set_label('% Change in Pedestrian Traffic', fontsize=10, weight='bold')
cbar.outline.set_visible(False)

# Set limits with a small buffer
buffer = 500
ax.set_xlim(plot_data['x'].min() - buffer, plot_data['x'].max() + buffer)
ax.set_ylim(plot_data['y'].min() - buffer, plot_data['y'].max() + buffer)

# Stops map from being stretched
ax.set_aspect('equal')

# Add basemap
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)
ax.set_axis_off()

# Add Title Box

# ax.text(0.05, 0.95, "Pedestrian Traffic\nin Melbourne", 
#         transform=ax.transAxes, 
#         fontsize=22, fontweight='bold', 
#         va='top', ha='left',
#         linespacing=1.2,
#         zorder=11)

# ax.text(0.05, 0.85, "Comparing 2019 to 2025", 
#         transform=ax.transAxes, 
#         fontsize=11, color='#555555',
#         va='top', ha='left',
#         zorder=11)

# ax.add_artist(patches.FancyBboxPatch(
#     (0.04, 0.82), 0.45, 0.14, # (x, y), width, height in axes fraction
#     transform=ax.transAxes,
#     boxstyle="round,pad=0.02",
#     facecolor='white', 
#     edgecolor='none', 
#     alpha=0.75, 
#     zorder=10
# ))

plt.tight_layout()
plt.show()

fig.savefig(config.WEBAPP_RESOURCES_DIR / "2dplot.svg", transparent=True, bbox_inches='tight')

In [ ]:
# Get RGB values at intermediate positions on the colormap [0, 1]
for pos in range(0, 10, 1):
    rgba = cmap(pos/10)
    rgb = [int(255*rgba[0]), int(255*rgba[1]), int(255*rgba[2])]
    print(f"Position {pos/10}: RGB={rgb}")

## Adding Colour

In [ ]:
pydeck_data = sensor_effects.copy()
pydeck_data['absolute_change'] = abs(pydeck_data['percentage_change'])

In [ ]:
import cmasher as cmr
import matplotlib.colors as mplcolors

# Apply colourmap to data

cmap = cmr.get_sub_cmap("cmr.prinsenvlag", 0.0, 1.0)
# cmap = cmap.reversed()

normalised_colour_map = mplcolors.TwoSlopeNorm(
    vmin=sensor_effects['percentage_change'].min(),
    vcenter=0,
    vmax=sensor_effects['percentage_change'].max()
)

def cmr_to_rgb(value):
    r, g, b, _ = cmap(normalised_colour_map(value))
    return [int(255*r), int(255*g), int(255*b)]

pydeck_data['colour'] = pydeck_data['percentage_change'].apply(cmr_to_rgb)

In [ ]:
def two_colour_map(pchange):
    if pchange > 0:
        return  [132, 202, 102]
    else:
        return [244, 109, 67]

pydeck_data['colour'] = pydeck_data['percentage_change'].apply(
    two_colour_map
)

In [ ]:
export_cols = [
    "longitude",
    "latitude",
    "percentage_change",
    "absolute_change",
    "sensor_description",
    "colour",
    "sensor_id"
]

pydeck_data[export_cols].to_json(
    "../webapp/resources/pedestrian_data.json",
    orient="records"
)

## Graphing Sensor Plots

In [ ]:
%matplotlib agg

import pandas as pd
import numpy as np
import matplotlib.dates as mdates

counts = pd.read_parquet(config.PROCESSED_DATA_DIR / 'analysis_data.parquet')
locations = pd.read_parquet(config.PROCESSED_LOCATIONS_FILE)

sensors = counts['sensor_id'].unique()

rgb_grn_frac = tuple(np.divide([132, 202, 102], 256))
rgb_blu_frac = tuple(np.divide([13, 110, 253], 256))

for s_id in sensors:
    
    sensor_location = locations[locations['sensor_id'] == s_id].squeeze()
    sensor_counts = counts[counts['sensor_id'] == s_id]

    fig, ax = plt.subplots(figsize=(12, 6))
    
    ax.set_title(f'Pedestrian Traffic at {sensor_location['sensor_description']}', fontsize=16, pad=20)

    sensor_counts_2019 = sensor_counts[sensor_counts['sensing_date'].dt.year == 2019]
    sensor_counts_2025 = sensor_counts[sensor_counts['sensing_date'].dt.year == 2025]

    ax.plot(
        sensor_counts_2019['sensing_date'].dt.day_of_year,
        sensor_counts_2019['daily_count'],
        color=rgb_blu_frac,
        alpha=0.8,
        linewidth=2.5,
        label='2019'
    )
    ax.plot(
        sensor_counts_2025['sensing_date'].dt.day_of_year,
        sensor_counts_2025['daily_count'],
        color=rgb_grn_frac,
        alpha=0.8,
        linewidth=2.5,
        label='2025'
    )
    
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.set_xlim(0, 370)
    ax.set_xlabel('Day of the year', fontsize=12)
    ax.set_ylabel('Daily pedestrian count', fontsize=12)

    ax.legend(fontsize=11, loc='upper left', framealpha=0.95)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()

    fig.savefig(config.WEBAPP_RESOURCES_DIR / 'sensorplots' / f"{s_id}_sensor_plot.svg", transparent=True, bbox_inches='tight')
